In [1]:
import os
import nltk
import json
from lib import StyloAnalyzer

corpora = ['aurthur_conan_doyle', 'charles_dickens',
           'h_g_wells', 'jane_austen', 'jonathan_swift']


In [2]:
csv_data = []
json_data = {}
for corpus in corpora:
    for entry in os.scandir(os.path.join('corpora_split', corpus)):
        if entry.is_file():
            with open(entry.path, 'r') as file:
                s = StyloAnalyzer(corpus + '.' + entry.name, file.read())
                csv_data.append(str(s))
                json_data[corpus + '.' + entry.name] = s.toJSON()
        # break


In [3]:
with open('output/result2.json', 'w') as file:
    file.write(json.dumps(json_data, indent=2))


In [4]:
csv_data.sort()
csv_data.insert(0, StyloAnalyzer.csv_header())
with open('output/result2.tsv', 'w') as file:
    file.write('\n'.join(csv_data))
